# Creating relations between Databases using SQLite3  using the Academy Awards dataset

This is a simple project to get started with creating one to many and many to many relations using Sqlite3 which a lightweight 
SQL library built into Python.

We will use the nominations.db (already created) database which contains the data found from here (https://www.aggdata.com/awards/oscar)

In [63]:
import sqlite3
conn = sqlite3.connect("nominations.db")

# Exploring the data

In [64]:
schema = conn.execute("PRAGMA table_info(nominations);").fetchall()
first_ten = conn.execute("SELECT * FROM nominations LIMIT 10;").fetchall()
for data in schema:
    print(data)
for data_ in first_ten:
    print(data_)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)
(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting 

We will create a new table called ceremonies which contains the Year and Host data. This is to avoid redundancy by
adding these columns to the nominations table. But first lets find the host and year information.

# Creating and populating new "Ceremonies" table.

In [34]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]
create_ceremonies = "CREATE TABLE ceremonies (Id INTEGER PRIMARY KEY,YEAR INTEGER,Host TEXT);"
conn.execute(create_ceremonies)
insert_query = "INSERT INTO ceremonies (Year,Host) VALUES (?,?);"
connn.executemany(insert_query,years_hosts)

OperationalError: table ceremonies already exists

We will ignore the above error

In [35]:
print(conn.execute("SELECT * FROM ceremonies LIMIT 10;").fetchall())
print(conn.execute("PRAGMA table_info(ceremonies);").fetchall())

[(1, 2010, 'Steve Martin'), (2, 2009, 'Hugh Jackman'), (3, 2008, 'Jon Stewart'), (4, 2007, 'Ellen DeGeneres'), (5, 2006, 'Jon Stewart'), (6, 2005, 'Chris Rock'), (7, 2004, 'Billy Crystal'), (8, 2003, 'Steve Martin'), (9, 2002, 'Whoopi Goldberg'), (10, 2001, 'Steve Martin')]
[(0, 'id', 'integer', 0, None, 1), (1, 'Year', 'integer', 0, None, 0), (2, 'Host', 'text', 0, None, 0)]


We can now see the Ceremonies table containing the Year and Host info. Now we will attempt to link this table 
to the nomination table by using Foreign Keys.

In [36]:
conn.execute("PRAGMA foreign_keys = ON;") #Turning on foreign key constraints

We need to delete the "Year" column from the nominations table. Unfortunately SQLite does not allow us to delete a column.
Instead we will do the following;
- Create a new table "nominations_two" with the schema we want.
- Populate nominations_two with the records we want.
- Delete the original nominations table.
- Rename nominations_two to nominations.

"nominations_two" will contain ID, category,nominee,movie,character,won, ceremony_id (as a foreign key to id column from ceremonies table).

We will also retrieve all the data from nominations table and store it in joined_nominations.

# Create "nominations_two" table and copy the data from nominations to nominations_two

In [37]:
create_nominations_two = "CREATE TABLE nominations_two (Id INTEGER PRIMARY KEY,Category TEXT,Nominee TEXT,Movie TEXT,Character TEXT,Won TEXT,Ceremony_id INTEGER,FOREIGN KEY (Ceremony_id) REFERENCES Ceremonies(Id));"
conn.execute(create_nominations_two)


OperationalError: table nominations_two already exists

In [42]:
data_nominations = "SELECT ceremonies.id as ceremony_id,nominations.Category,nominations.Nominee,nominations.Movie,nominations.Character,nominations.Won FROM nominations INNER JOIN ceremonies ON nominations.Year == ceremonies.year;"
joined_nominations = conn.execute(data_nominations).fetchall()

In [43]:
insert_query_nominations_two = "INSERT INTO nominations_two (ceremony_id,Category,Nominee,Movie,Character,Won) VALUES (?,?,?,?,?,?);"
conn.executemany(insert_query_nominations_two,joined_nominations)
print(conn.execute("SELECT * FROM nominations_two LIMIT 5").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', '0', 1), (2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', '0', 1), (3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', '0', 1), (4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', '1', 1), (5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', '0', 1)]


We can now see that the data from nominations has been passed into nomination_two along with the foreign key column
ceremony_id which links to ceremonies_table. We will now delete the nominations table and rename nominations_two to nominations.

# Deleting "nominations" table and renaming "nominations_two" to "nominations"

In [44]:
delete_nominations = "DROP TABLE nominations;"
conn.execute(delete_nominations)
rename_nominations_two = "ALTER TABLE nominations_two RENAME to nominations"
conn.execute(rename_nominations_two)

# Creating 3 new tables for a Many-Many relation type normalization.
- Movies Table
- Actors Table
- Movies_Actors Table

In [45]:
create_movies = "CREATE TABLE movies (Id INTEGER PRIMARY KEY,Movies TEXT);"
conn.execute(create_movies)
create_actors = "CREATE TABLE actors (Id INTEGER PRIMARY KEY,Actor TEXT);"
conn.execute(create_actors)


In [46]:
create_movies_actors = "CREATE TABLE movies_actors (Id INTEGER PRIMARY KEY,movie_Id INTEGER REFERENCES movies(Id),actor_id INTEGER REFERENCES actors(Id));"
conn.execute(create_movies_actors)

# Populating the Movies and Actors tables

In [50]:
insert_movies = "INSERT INTO movies (Movies) SELECT DISTINCT movie FROM nominations;"
conn.execute(insert_movies)
insert_actor = "INSERT INTO actors (Actor) SELECT DISTINCT nominee FROM nominations;"
conn.execute(insert_actor)
print(conn.execute("select * from movies limit 5;").fetchall())
print(conn.execute("select * from actors limit 5;").fetchall())

[(1, 'Biutiful'), (2, 'True Grit'), (3, 'The Social Network'), (4, "The King's Speech"), (5, '127 Hours')]
[(1, 'Javier Bardem'), (2, 'Jeff Bridges'), (3, 'Jesse Eisenberg'), (4, 'Colin Firth'), (5, 'James Franco')]


# Populating the movies_actors table

In [62]:
get_movies_actors = "SELECT movie, nominee FROM nominations"
get_movies_actors = conn.execute(get_movies_actors).fetchall()
insert_movies_actors = "insert into movies_actors (movie_id, actor_id) values ((select id from movies where Movies == ?),(select id from actors where Actor == ?));"
conn.executemany(insert_movies_actors, get_movies_actors)
print(conn.execute("select * from movies_actors limit 5;").fetchall())

[(1, 1, 1), (2, 2, 2), (3, 3, 3), (4, 4, 4), (5, 5, 5)]
